*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
vocab = {}  
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ") 
  encoding = []  

  for word in words:
    if word in vocab:
      code = vocab[word]  
      encoding.append(code) 
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1
  
  return encoding

In [ ]:
train = pd.read_csv(train_file_path, sep='\t', header=None)
test = pd.read_csv(test_file_path, sep='\t', header=None)

In [ ]:
train["encoding"] = [one_hot_encoding(train.iloc[i,1]) for i in range(4179)]
test["encoding"] = [one_hot_encoding(test.iloc[i,1]) for i in range(1392)]

In [ ]:
train.columns = ["ham/spam", "message", "encoding"]
test.columns = ["ham/spam", "message", "encoding"]

In [ ]:
train.pop("message")
test.pop("message")

In [ ]:
train_labels = train.pop("ham/spam")
test_labels = test.pop("ham/spam")

In [ ]:
ar = []
for i in train_labels.values:
  if i == "ham":
    ar.append(0)
  else:
    ar.append(1)
train_labels_encoding = np.array(ar)

ar1 = []
for i in test_labels.values:
  if i == "ham":
    ar1.append(0)
  else:
    ar1.append(1)
test_labels_encoding = np.array(ar1)

In [ ]:
train

In [ ]:
test

In [ ]:
train_encoding = train["encoding"].tolist()
test_encoding = test["encoding"].tolist()

In [ ]:
train_encoding = np.asarray(train_encoding)
test_encoding = np.asarray(test_encoding)

In [ ]:
train_encoding[1]

In [ ]:
test_encoding[1]

In [ ]:
train_labels_encoding

In [ ]:
test_labels_encoding

In [ ]:
train_encoding = tf.keras.preprocessing.sequence.pad_sequences(train_encoding, 250)
test_encoding = tf.keras.preprocessing.sequence.pad_sequences(test_encoding, 250)

In [ ]:
VOCAB_SIZE = 88584
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [ ]:
history = model.fit( tf.convert_to_tensor(train_encoding),  tf.convert_to_tensor(train_labels_encoding), epochs=10, validation_split=0.2)

In [ ]:
results = model.evaluate(test_encoding, test_labels_encoding)
print(results)

In [ ]:
def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [vocab[word] if word in vocab else 0 for word in tokens]
  return tf.keras.preprocessing.sequence.pad_sequences([tokens], 250)[0]
encode_text("We are same")

In [ ]:
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  return result[0][0]
predict("Mobile sale is live")

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = []
  prediction.append(predict(pred_text))
  if round(predict(pred_text)) == 0:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
